## Random Forest-Mnist

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump
import numpy as np
import xgboost as xgb
import os
from sklearn.preprocessing import LabelEncoder
import threading
import time
import subprocess
import psutil
import tempfile
import re
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
from sklearn.pipeline import Pipeline
from nyoka import skl_to_pmml
from nyoka import PMML44 as pml
from pypmml import Model
import joblib
from onnxconverter_common import FloatTensorType



2024-08-08 15:25:50.001757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import joblib
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.datasets import mnist
from scipy.sparse import hstack

In [3]:
# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将图像数据从 3D 重塑为 2D (每个图像为一行，像素为列)
X_train_2d = X_train.reshape(-1, 28*28)
X_test_2d = X_test.reshape(-1, 28*28)

# 将数据类型转换为 float32 并进行归一化处理
X_train_2d = X_train_2d.astype('float32') / 255
X_test_2d = X_test_2d.astype('float32') / 255

In [5]:
# 创建随机森林分类器
# n_estimators 表示使用的决策树数量，更多的树会提高模型的性能和稳定性，但也会增加计算成本
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
clf.fit(X_train_2d, y_train)

# 预测测试集
y_pred = clf.predict(X_test_2d)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# 保存训练好的模型到文件
model_filename = ('./models_train/mnist_random_forest_model.joblib')
dump(clf, model_filename)
print(f"Model saved to {model_filename}")

Test accuracy: 97.04%
Model saved to ./models_train/mnist_random_forest_model.joblib


## RF-Mnist-jo

In [6]:
import joblib
# 初始化全局变量    
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_tfidf):
    global duration
    global inference_duration

    start_time = time.time()
    for i in range(10):
        joblib.dump(model, 'mnist_random_forest_model.joblib')
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()
    # 进行推理
    for i in range(1): # 1*10
        model.predict(X_test_tfidf)
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载RF模型
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-joblib.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
#print(filtered_lines_count)
#print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Subprocess started.
Time taken to save model: 1.2319 seconds
Time taken for inference on samples: 0.3262 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_rf/output-mnist-rf-joblib.txt
116
['CPU Power: 15704 mW', 'GPU Power: 0 mW', 'CPU Power: 20359 mW', 'GPU Power: 0 mW', 'CPU Power: 15190 mW', 'GPU Power: 0 mW', 'CPU Power: 19905 mW', 'GPU Power: 0 mW', 'CPU Power: 21114 mW', 'GPU Power: 0 mW', 'CPU Power: 21789 mW', 'GPU Power: 61 mW', 'CPU Power: 16594 mW', 'GPU Power: 0 mW', 'CPU Power: 9221 mW', 'GPU Power: 68 mW', 'CPU Power: 13149 mW', 'GPU Power: 0 mW', 'CPU Power: 15338 mW', 'GPU Power: 0 mW', 'CPU Power: 17272 mW', 'GPU Power: 0 mW', 'CPU Power: 19868 mW', 'GPU Power: 64 mW', 'CPU Power: 16448 mW', 'GPU Power: 64 mW', 'CPU Power: 17779 mW', 'GPU Power: 0 mW', 'CPU Power: 15277 mW', 'GPU Power: 164 mW', 'CPU Power: 18224 mW', 'GPU Power: 441 mW', 'CPU Power: 12106 mW', 'GPU Power: 254 mW', 'CPU Power: 13442 mW', 'GPU Power: 0 mW

In [10]:
import joblib
import time
import threading
import psutil
import subprocess
import tempfile
import os
import numpy as np
import re

# 初始化全局变量    
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_tfidf):
    global duration
    global inference_duration

    start_time = time.time()
    for i in range(5):
        joblib.dump(model, 'mnist_random_forest_model.joblib')
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()

    # 批量推理
    batch_size = 128
    num_samples = len(X_test_tfidf)
    num_batches = (num_samples + batch_size - 1) // batch_size  # 计算批次数量

    for _ in range(1):  # 推理循环次数为 1
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, num_samples)
            batch = X_test_tfidf[batch_start:batch_end]
            model.predict(batch)
    
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载RF模型
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')

# 假设 X_test_2d 是测试数据
X_test_2d = np.random.rand(10000, 28*28)  # 示例数据

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-joblib.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
#print(filtered_lines_count)
#print(filtered_lines)


duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")


Subprocess started.
Time taken to save model: 0.5539 seconds
Time taken for inference on samples: 0.6868 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_rf/output-mnist-rf-joblib.txt
Total energy consumption: 3745.28 mV


### RF-MNIST-ONNX

In [12]:
# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_2d):
    global duration
    global inference_duration

    start_time = time.time()
    for i in range(1):
        # 转换为ONNX格式
        initial_type = [('float_input', FloatTensorType([None, X_test_2d.shape[1]]))]
        # onnx_model = convert_sklearn(model, initial_types=initial_type, target_opset=9)

        # 保存ONNX模型
        onnx_model_filename = 'mnist_random_forest_model.onnx'
        # with open(onnx_model_filename, "wb") as f:
        #     f.write(onnx_model.SerializeToString())
        print(f"ONNX model saved to {onnx_model_filename}")
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()
    X_test_2d_array = X_test_2d.astype(np.float32)

    # 使用ONNX runtime进行推理
    sess = rt.InferenceSession(onnx_model_filename)
    # sess = rt.InferenceSession(onnx_model.SerializeToString())
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name

    # 批量推理
    batch_size = 128
    num_samples = len(X_test_2d_array)
    num_batches = (num_samples + batch_size - 1) // batch_size  # 计算批次数量

    for _ in range(1):  # 推理循环次数为 1
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, num_samples)
            batch = X_test_2d_array[batch_start:batch_end]
            predictions = sess.run([label_name], {input_name: batch})[0]
            # print(predictions)
    
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {num_samples} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载RF模型
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')

# 假设 X_test_2d 是测试数据
X_test_2d = np.random.rand(10000, 28*28)  # 示例数据

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-onnx.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
# print(filtered_lines_count)
# print(filtered_lines)

duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")


ONNX model saved to mnist_random_forest_model.onnx
Time taken to save model: 0.0005 seconds
Subprocess started.
Time taken for inference on 10000 samples: 1.7049 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_rf/output-mnist-rf-onnx.txt
Total energy consumption: 10860.29 mV


### RF MNIST PTH

In [13]:
import threading
import time
import numpy as np
import joblib
import psutil
import subprocess
import tempfile
import os
import torch
from hummingbird.ml import convert
from sklearn.ensemble import RandomForestClassifier

# Initialize global variables
thread_output = {}
duration = 0
inference_duration = 0

def save_model(stop_event, model, X_test_2d):
    global duration
    global inference_duration

    start_time = time.time()
    # Convert the RandomForest model to a PyTorch model using Hummingbird
    pytorch_model = convert(model, 'pytorch')
    
    # Save the PyTorch model
    torch.save(pytorch_model, 'mnist_random_forest_model.pth')
    print("PyTorch model saved to mnist_random_forest_model.pth")

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    # Perform inference using PyTorch
    start_time_inference = time.time()
    X_test_tensor = torch.tensor(X_test_2d.astype(np.float32))
    batch_size = 128
    num_samples = X_test_tensor.shape[0]
    num_batches = (num_samples + batch_size - 1) // batch_size

    # Use no .eval() call as Hummingbird models do not require it
    predictions = []
    with torch.no_grad():
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, num_samples)
            batch = X_test_tensor[batch_start:batch_end]
            predictions.extend(pytorch_model.predict(batch))

    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {num_samples} samples: {inference_duration:.4f} seconds')

    stop_event.set()

def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)

    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:
                    break
            if process.poll() is None:
                process.terminate()
                process.wait(timeout=0.1)
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')
X_test_2d = np.random.rand(10000, 28*28)  # Example data

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

thread1.join()
thread2.join()
thread3.join()

content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-pytorch.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)

duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")


Subprocess started.
PyTorch model saved to mnist_random_forest_model.pth
Time taken to save model: 9.1638 seconds
Time taken for inference on 10000 samples: 2.8768 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_rf/output-mnist-rf-pytorch.txt
Total energy consumption: 14425.17 mV


### RF-Mnist-pmml

In [13]:
# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_2d):
    global duration
    global inference_duration

    start_time = time.time()
    for i in range(1):
        # 创建一个Pipeline
        pipeline = Pipeline([("classifier", model)])

        # 导出为PMML
        pmml_filename = 'mnist_random_forest_model.pmml'
        skl_to_pmml(pipeline, col_names=[f"feature_{i}" for i in range(X_test_2d.shape[1])], target_name="digit", pmml_f_name=pmml_filename)
    print(f"Model exported to {pmml_filename}")
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    start_time_inference = time.time()
    # 加载PMML模型
    model = Model.load(pmml_filename)

    # 准备输入数据
    input_vector = X_test_2d[:1]

    # 进行推理
    for _ in range(1):
        y_pred = model.predict(input_vector)
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {input_vector.shape[0]} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载RF模型
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-pmml.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")


Subprocess started.
Model exported to mnist_random_forest_model.pmml
Time taken to save model: 154.2145 seconds
Time taken for inference on 1 samples: 44.9228 seconds
Resource monitoring finished.
Subprocess finished.
Content saved to ./mnist_models/output_rf/output-mnist-rf-pmml.txt
19834
['CPU Power: 8134 mW', 'GPU Power: 159 mW', 'CPU Power: 6675 mW', 'GPU Power: 155 mW', 'CPU Power: 5622 mW', 'GPU Power: 76 mW', 'CPU Power: 7711 mW', 'GPU Power: 40 mW', 'CPU Power: 5939 mW', 'GPU Power: 0 mW', 'CPU Power: 5607 mW', 'GPU Power: 45 mW', 'CPU Power: 4908 mW', 'GPU Power: 93 mW', 'CPU Power: 4419 mW', 'GPU Power: 0 mW', 'CPU Power: 4905 mW', 'GPU Power: 58 mW', 'CPU Power: 9490 mW', 'GPU Power: 0 mW', 'CPU Power: 8813 mW', 'GPU Power: 0 mW', 'CPU Power: 7498 mW', 'GPU Power: 0 mW', 'CPU Power: 6841 mW', 'GPU Power: 0 mW', 'CPU Power: 7615 mW', 'GPU Power: 0 mW', 'CPU Power: 7084 mW', 'GPU Power: 0 mW', 'CPU Power: 5131 mW', 'GPU Power: 0 mW', 'CPU Power: 6748 mW', 'GPU Power: 0 mW', 'C

In [17]:
# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义保存模型的函数
def save_model(stop_event, model, X_test_2d):
    global duration
    global inference_duration

    start_time = time.time()
    for i in range(1):
        # 创建一个Pipeline
        pipeline = Pipeline([("classifier", model)])

        # 导出为PMML
        pmml_filename = 'mnist_random_forest_model.pmml'
        #skl_to_pmml(pipeline, col_names=[f"feature_{i}" for i in range(X_test_2d.shape[1])], target_name="digit", pmml_f_name=pmml_filename)
    print(f"Model exported to {pmml_filename}")
    end_time = time.time()

    duration = end_time - start_time
    print(f'Time taken to save model: {duration:.4f} seconds')

    # 加载PMML模型
    model = Model.load(pmml_filename)

    # 准备输入数据
    X_test_2d_array = X_test_2d.astype(np.float32)

    # 批量推理
    batch_size = 128
    num_samples = len(X_test_2d_array)
    num_batches = (num_samples + batch_size - 1) // batch_size  # 计算批次数量

    start_time_inference = time.time()
    for _ in range(1):  # 推理循环次数为 1
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, num_samples)
            batch = X_test_2d_array[batch_start:batch_end]
            y_pred = model.predict(batch)
    
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {num_samples} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()

# 加载RF模型
loaded_model = joblib.load('./models_train/mnist_random_forest_model.joblib')

# 假设 X_test_2d 是测试数据
X_test_2d = np.random.rand(1000, 28*28)  # 示例数据

thread1 = threading.Thread(target=save_model, args=(stop_event, loaded_model, X_test_2d))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

output_file_name = './mnist_models/output_rf/output-mnist-rf-pmml.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
#print(filtered_lines_count)
#print(filtered_lines)

duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")


Model exported to mnist_random_forest_model.pmml
Time taken to save model: 0.0004 seconds
Subprocess started.
Time taken for inference on 1000 samples: 86.6784 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_rf/output-mnist-rf-pmml.txt
Total energy consumption: 675943.92 mV
